In [ ]:
#@title 🎧 Download Narration Audio & Play Introduction
import os as _os
if not _os.path.exists("/content/narration"):
    !pip install -q gdown
    import gdown
    gdown.download(id="17rFuCNZUUY1xHrMq1WTamV-JWh_IDZe8", output="/content/narration.zip", quiet=False)
    !unzip -q /content/narration.zip -d /content/narration
    !rm /content/narration.zip
    print(f"Loaded {len(_os.listdir('/content/narration'))} narration segments")
else:
    print("Narration audio already loaded.")

from IPython.display import Audio, display
display(Audio("/content/narration/04_00_intro.mp3"))


In [ ]:
# 🔧 Setup: Run this cell first!
# Check GPU availability and install dependencies

import torch
import sys

# Check GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"✅ GPU available: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    device = torch.device('cpu')
    print("⚠️ No GPU detected. Some cells may run slowly.")
    print("   Go to Runtime → Change runtime type → GPU")

print(f"\n📦 Python {sys.version.split()[0]}")
print(f"🔥 PyTorch {torch.__version__}")

# Set random seeds for reproducibility
import random
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print(f"🎲 Random seed set to {SEED}")

%matplotlib inline

In [ ]:
#@title 🎧 Listen: Motivation Intuition
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_02_motivation_intuition.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


# Sampling, DDIM, and Score Connection — Vizuara

## 1. Why Does This Matter?

We have built and trained a complete DDPM. But there are two critical topics we have not explored yet. First, DDPM sampling requires 1000 neural network evaluations per image — can we do it faster? Second, how does noise prediction connect to the broader world of score-based generative models?

In this notebook, we will:
- Analyze the DDPM sampling algorithm in detail
- Implement DDIM (Denoising Diffusion Implicit Models) for 10-50x faster sampling
- Derive the connection between noise prediction and score functions
- Implement Langevin dynamics sampling to see the score connection in action
- Compare DDPM vs DDIM vs Langevin sampling quality

## 2. Building Intuition

Imagine you are a sculptor. DDPM sampling is like carving a statue from a block of marble using 1000 tiny chisel taps — each tap removes a tiny bit of material. The result is beautiful, but it takes forever.

DDIM is like using bigger chisel strokes — you take 50 well-placed cuts instead of 1000 tiny ones. The result is almost as good, but 20x faster.

Score-based sampling is a different technique entirely — instead of chiseling, you use magnets. You place magnets at the locations where the statue should be, and the marble dust naturally arranges itself into the right shape. Remarkably, this is mathematically equivalent to what DDPM is doing.

In [ ]:
#@title 🎧 Code Walkthrough: Lib Params Setup
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_03_lib_params_setup.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
!pip install torch torchvision matplotlib numpy -q

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import math
import time

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Diffusion parameters
T = 1000
betas = torch.linspace(1e-4, 0.02, T).to(device)
alphas = (1.0 - betas).to(device)
alpha_bars = torch.cumprod(alphas, dim=0).to(device)

In [ ]:
#@title 🎧 Code Walkthrough: Unet Model Def
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_04_unet_model_def.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Load a pretrained model (or use the one from Notebook 3)
# For this notebook, we provide a minimal U-Net and pretrain it quickly

class SinusoidalTimeEmbedding(nn.Module):
    def __init__(self, dim):
        super().__init__()
        self.dim = dim
    def forward(self, t):
        half_dim = self.dim // 2
        emb = math.log(10000) / (half_dim - 1)
        emb = torch.exp(torch.arange(half_dim, device=t.device) * -emb)
        emb = t[:, None].float() * emb[None, :]
        return torch.cat([torch.sin(emb), torch.cos(emb)], dim=-1)

class ResBlock(nn.Module):
    def __init__(self, in_ch, out_ch, time_dim):
        super().__init__()
        self.conv1 = nn.Conv2d(in_ch, out_ch, 3, padding=1)
        self.conv2 = nn.Conv2d(out_ch, out_ch, 3, padding=1)
        self.time_mlp = nn.Linear(time_dim, out_ch)
        self.norm1 = nn.GroupNorm(8, out_ch)
        self.norm2 = nn.GroupNorm(8, out_ch)
        self.shortcut = nn.Conv2d(in_ch, out_ch, 1) if in_ch != out_ch else nn.Identity()
    def forward(self, x, t_emb):
        h = self.norm1(F.relu(self.conv1(x)))
        h = h + F.relu(self.time_mlp(t_emb))[:, :, None, None]
        h = self.norm2(F.relu(self.conv2(h)))
        return h + self.shortcut(x)

class UNet(nn.Module):
    def __init__(self, in_ch=1, base_ch=64, time_dim=256):
        super().__init__()
        self.time_embed = nn.Sequential(
            SinusoidalTimeEmbedding(base_ch), nn.Linear(base_ch, time_dim),
            nn.ReLU(), nn.Linear(time_dim, time_dim))
        self.enc1 = ResBlock(in_ch, base_ch, time_dim)
        self.enc2 = ResBlock(base_ch, base_ch*2, time_dim)
        self.enc3 = ResBlock(base_ch*2, base_ch*4, time_dim)
        self.down1 = nn.Conv2d(base_ch, base_ch, 4, 2, 1)
        self.down2 = nn.Conv2d(base_ch*2, base_ch*2, 4, 2, 1)
        self.down3 = nn.Conv2d(base_ch*4, base_ch*4, 4, 2, 1)
        self.bottleneck = ResBlock(base_ch*4, base_ch*4, time_dim)
        self.up3 = nn.ConvTranspose2d(base_ch*4, base_ch*4, 4, 2, 1)
        self.up2 = nn.ConvTranspose2d(base_ch*2, base_ch*2, 4, 2, 1)
        self.up1 = nn.ConvTranspose2d(base_ch, base_ch, 4, 2, 1)
        self.dec3 = ResBlock(base_ch*8, base_ch*2, time_dim)
        self.dec2 = ResBlock(base_ch*4, base_ch, time_dim)
        self.dec1 = ResBlock(base_ch*2, base_ch, time_dim)
        self.final = nn.Conv2d(base_ch, in_ch, 1)

    def forward(self, x, t):
        t_emb = self.time_embed(t)
        e1 = self.enc1(x, t_emb)
        e2 = self.enc2(self.down1(e1), t_emb)
        e3 = self.enc3(self.down2(e2), t_emb)
        b = self.bottleneck(self.down3(e3), t_emb)
        d3 = self.dec3(torch.cat([self.up3(b), e3], 1), t_emb)
        d2 = self.dec2(torch.cat([self.up2(d3), e2], 1), t_emb)
        d1 = self.dec1(torch.cat([self.up1(d2), e1], 1), t_emb)
        return self.final(d1)

model = UNet().to(device)
print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")

In [ ]:
#@title 🎧 Listen: Math Ddpm Review
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_05_math_ddpm_review.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
#@title 🎧 Listen: Math Ddim Sampling
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_06_math_ddim_sampling.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
#@title 🎧 Listen: Math Score Connection
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_07_math_score_connection.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 3. The Mathematics

### DDPM Sampling (Review)

The DDPM sampling rule at each step is:

$$x_{t-1} = \frac{1}{\sqrt{\alpha_t}}\left(x_t - \frac{\beta_t}{\sqrt{1-\bar{\alpha}_t}} \epsilon_\theta(x_t, t)\right) + \sigma_t z$$

where $z \sim \mathcal{N}(0, I)$ and $\sigma_t = \sqrt{\beta_t}$.

Let us plug in numbers at $t=500$: $\alpha_t=0.99$, $\beta_t=0.01$, $\bar{\alpha}_t=0.5$.
If $x_t = 0.4$ and $\epsilon_\theta = 0.3$, $z = 0.1$:
- $x_{t-1} = \frac{1}{0.995}(0.4 - \frac{0.01}{0.707} \times 0.3) + 0.1 \times 0.1 = 1.005 \times 0.396 + 0.01 = 0.408$

### DDIM Sampling

DDIM removes the stochastic noise term, making sampling **deterministic**:

$$x_{t-1} = \sqrt{\bar{\alpha}_{t-1}}\left(\frac{x_t - \sqrt{1-\bar{\alpha}_t}\epsilon_\theta}{\sqrt{\bar{\alpha}_t}}\right) + \sqrt{1-\bar{\alpha}_{t-1}} \cdot \epsilon_\theta$$

The key insight: DDIM can skip timesteps! Instead of sampling at every $t \in \{1000, 999, ..., 1\}$, we can use a subsequence like $\{1000, 980, 960, ..., 20\}$ (50 steps instead of 1000).

### The Score Connection

The noise prediction and score function are related by:

$$\epsilon_\theta(x_t, t) = -\sqrt{1-\bar{\alpha}_t} \cdot \nabla_{x_t} \log q(x_t)$$

This means predicting noise is equivalent to estimating the score (gradient of log-probability). The DDPM sampling update is actually discretized Langevin dynamics!

## 4. Let's Build It — Component by Component

### Step 1: DDPM Sampler

In [ ]:
#@title 🎧 Code Walkthrough: Ddpm Sampler Impl
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_08_ddpm_sampler_impl.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
@torch.no_grad()
def sample_ddpm(model, n_samples=16, img_shape=(1, 32, 32)):
    """Standard DDPM sampling: 1000 steps."""
    model.eval()
    x = torch.randn(n_samples, *img_shape, device=device)

    for t in reversed(range(T)):
        t_batch = torch.full((n_samples,), t, device=device, dtype=torch.long)
        eps = model(x, t_batch)

        alpha_t = alphas[t]
        alpha_bar_t = alpha_bars[t]
        beta_t = betas[t]

        x = (1 / torch.sqrt(alpha_t)) * (x - beta_t / torch.sqrt(1 - alpha_bar_t) * eps)
        if t > 0:
            x += torch.sqrt(beta_t) * torch.randn_like(x)

    return x

In [ ]:
#@title 🎧 Code Walkthrough: Ddim Sampler Impl
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_09_ddim_sampler_impl.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### Step 2: DDIM Sampler

In [ ]:
@torch.no_grad()
def sample_ddim(model, n_samples=16, img_shape=(1, 32, 32), num_steps=50):
    """
    DDIM sampling: deterministic, with configurable number of steps.
    Much faster than DDPM while maintaining quality.
    """
    model.eval()
    x = torch.randn(n_samples, *img_shape, device=device)

    # Create a subsequence of timesteps
    step_size = T // num_steps
    timesteps = list(range(0, T, step_size))
    timesteps = list(reversed(timesteps))

    for i in range(len(timesteps) - 1):
        t = timesteps[i]
        t_prev = timesteps[i + 1]

        t_batch = torch.full((n_samples,), t, device=device, dtype=torch.long)
        eps = model(x, t_batch)

        alpha_bar_t = alpha_bars[t]
        alpha_bar_t_prev = alpha_bars[t_prev]

        # DDIM update (deterministic — no random noise added)
        x0_pred = (x - torch.sqrt(1 - alpha_bar_t) * eps) / torch.sqrt(alpha_bar_t)
        x = (torch.sqrt(alpha_bar_t_prev) * x0_pred
             + torch.sqrt(1 - alpha_bar_t_prev) * eps)

    return x

In [ ]:
#@title 🎧 Code Walkthrough: Quick Training
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_10_quick_training.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Quick training (5 epochs for demonstration)
transform = transforms.Compose([
    transforms.Pad(2),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
dataset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True)

optimizer = torch.optim.Adam(model.parameters(), lr=2e-4)
print("Quick training (5 epochs)...")
for epoch in range(5):
    model.train()
    total_loss = 0
    for batch_idx, (images, _) in enumerate(dataloader):
        images = images.to(device)
        t = torch.randint(0, T, (images.shape[0],), device=device)
        noise = torch.randn_like(images)
        sqrt_ab = torch.sqrt(alpha_bars[t]).view(-1,1,1,1)
        sqrt_1_ab = torch.sqrt(1-alpha_bars[t]).view(-1,1,1,1)
        x_t = sqrt_ab * images + sqrt_1_ab * noise
        pred = model(x_t, t)
        loss = F.mse_loss(pred, noise)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"  Epoch {epoch+1}: Loss = {total_loss / len(dataloader):.4f}")

In [ ]:
#@title 🎧 Narration: Todo1 Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_11_todo1_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


**Checkpoint:** The loss should decrease over epochs, indicating the model is learning to predict noise.

## 5. Your Turn

### TODO 1: Implement Eta-Controlled DDIM

DDIM has a parameter $\eta$ that interpolates between deterministic ($\eta=0$) and stochastic ($\eta=1$, equivalent to DDPM) sampling.

In [ ]:
#@title 🎧 Before You Start: Todo1 Impl
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_12_todo1_impl.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
@torch.no_grad()
def sample_ddim_eta(model, n_samples=16, img_shape=(1, 32, 32), num_steps=50, eta=0.0):
    """
    TODO: Implement DDIM with controllable stochasticity via eta.

    When eta=0: deterministic (standard DDIM)
    When eta=1: stochastic (equivalent to DDPM)

    The update rule with eta is:
    sigma_t = eta * sqrt((1-alpha_bar_{t-1})/(1-alpha_bar_t)) * sqrt(beta_t)
    x_{t-1} = sqrt(alpha_bar_{t-1}) * x0_pred
            + sqrt(1 - alpha_bar_{t-1} - sigma_t^2) * eps_pred
            + sigma_t * z  (where z ~ N(0,I))

    Returns: generated samples
    """
    # HINT: Start with the DDIM code above and add the sigma_t term

    pass  # YOUR CODE HERE

In [ ]:
#@title 🎧 Narration: Todo2 Intro
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_13_todo2_intro.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


### TODO 2: Compare Sampling Speed and Quality

In [ ]:
#@title 🎧 Before You Start: Todo2 Impl
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_14_todo2_impl.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
def compare_samplers(model, num_ddim_steps_list=[10, 25, 50, 100, 200]):
    """
    TODO: Compare DDPM vs DDIM at different step counts.

    1. For each method, time the sampling process
    2. Generate 16 samples with each method
    3. Display results in a grid with timing information
    4. Discuss: at what point does DDIM quality match DDPM?

    Expected: DDIM with 50 steps should produce quality close to DDPM with 1000.
    """
    # HINT: Use time.time() to measure wall-clock time

    pass  # YOUR CODE HERE

In [ ]:
#@title 🎧 Code Walkthrough: Speed Comparison Run
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_15_speed_comparison_run.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 6. Putting It All Together

### Speed Comparison

In [ ]:
# Compare DDPM (1000 steps) vs DDIM (50 steps)
print("Sampling with DDPM (1000 steps)...")
start = time.time()
ddpm_samples = sample_ddpm(model, n_samples=8)
ddpm_time = time.time() - start
print(f"  DDPM time: {ddpm_time:.1f}s")

print("Sampling with DDIM (50 steps)...")
start = time.time()
ddim_samples = sample_ddim(model, n_samples=8, num_steps=50)
ddim_time = time.time() - start
print(f"  DDIM time: {ddim_time:.1f}s")

print(f"\nSpeedup: {ddpm_time / ddim_time:.1f}x faster!")

In [ ]:
# Visualize side by side
fig, axes = plt.subplots(2, 8, figsize=(24, 6))
for i in range(8):
    axes[0][i].imshow(ddpm_samples[i].squeeze().cpu().clamp(-1,1).numpy(), cmap='gray')
    axes[0][i].axis('off')
    if i == 0:
        axes[0][i].set_ylabel('DDPM\n(1000 steps)', fontsize=12, rotation=0, labelpad=80)

    axes[1][i].imshow(ddim_samples[i].squeeze().cpu().clamp(-1,1).numpy(), cmap='gray')
    axes[1][i].axis('off')
    if i == 0:
        axes[1][i].set_ylabel('DDIM\n(50 steps)', fontsize=12, rotation=0, labelpad=80)

plt.suptitle(f'DDPM ({ddpm_time:.1f}s) vs DDIM ({ddim_time:.1f}s) — {ddpm_time/ddim_time:.0f}x Speedup', fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
#@title 🎧 What to Look For: Score Connection Viz
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_16_score_connection_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 7. Training and Results

### The Score Function Connection

In [ ]:
# Demonstrate the connection: epsilon prediction = scaled score
def visualize_score_connection(model, x_0, t_val=500):
    """
    Show that noise prediction is equivalent to score estimation.

    score = -epsilon_theta / sqrt(1 - alpha_bar_t)
    """
    model.eval()
    x_0 = x_0.to(device)
    t = torch.tensor([t_val], device=device)

    noise = torch.randn_like(x_0)
    sqrt_ab = torch.sqrt(alpha_bars[t_val])
    sqrt_1_ab = torch.sqrt(1 - alpha_bars[t_val])
    x_t = sqrt_ab * x_0 + sqrt_1_ab * noise

    with torch.no_grad():
        eps_pred = model(x_t, t)

    # Score = -eps / sqrt(1 - alpha_bar_t)
    score = -eps_pred / sqrt_1_ab

    fig, axes = plt.subplots(1, 4, figsize=(16, 4))
    axes[0].imshow(x_0.squeeze().cpu().numpy(), cmap='gray')
    axes[0].set_title('Original x₀')
    axes[1].imshow(x_t.squeeze().cpu().clamp(-1,1).numpy(), cmap='gray')
    axes[1].set_title(f'Noisy x_t (t={t_val})')
    axes[2].imshow(eps_pred.squeeze().cpu().numpy(), cmap='RdBu')
    axes[2].set_title('Predicted Noise ε̂')
    axes[3].imshow(score.squeeze().cpu().numpy(), cmap='RdBu')
    axes[3].set_title('Score ∇log q(x_t)\n= -ε̂ / √(1-ᾱ_t)')
    for ax in axes:
        ax.axis('off')
    plt.suptitle('Noise Prediction ↔ Score Function', fontsize=14)
    plt.tight_layout()
    plt.show()

sample = dataset[0][0].unsqueeze(0)
visualize_score_connection(model, sample, t_val=300)

In [ ]:
#@title 🎧 What to Look For: Ddim Steps Viz
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_17_ddim_steps_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


In [ ]:
# Show DDIM at different step counts
fig, axes = plt.subplots(4, 8, figsize=(24, 12))
for row, num_steps in enumerate([10, 25, 50, 200]):
    samples = sample_ddim(model, n_samples=8, num_steps=num_steps)
    for col in range(8):
        axes[row][col].imshow(samples[col].squeeze().cpu().clamp(-1,1).numpy(), cmap='gray')
        axes[row][col].axis('off')
    axes[row][0].set_ylabel(f'{num_steps} steps', fontsize=12, rotation=0, labelpad=60)

plt.suptitle('DDIM Sampling Quality at Different Step Counts', fontsize=16)
plt.tight_layout()
plt.show()

In [ ]:
#@title 🎧 What to Look For: Final Output
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_18_final_output.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 8. Final Output

In [ ]:
# Generate the best samples we can
print("Generating final showcase samples...")
final_ddpm = sample_ddpm(model, n_samples=16)
final_ddim = sample_ddim(model, n_samples=16, num_steps=50)

fig, axes = plt.subplots(2, 16, figsize=(32, 4))
for i in range(16):
    axes[0][i].imshow(final_ddpm[i].squeeze().cpu().clamp(-1,1).numpy(), cmap='gray')
    axes[0][i].axis('off')
    axes[1][i].imshow(final_ddim[i].squeeze().cpu().clamp(-1,1).numpy(), cmap='gray')
    axes[1][i].axis('off')
axes[0][0].set_ylabel('DDPM', fontsize=12, rotation=0, labelpad=40)
axes[1][0].set_ylabel('DDIM', fontsize=12, rotation=0, labelpad=40)
plt.suptitle('Final Generated Samples: DDPM (1000 steps) vs DDIM (50 steps)', fontsize=16)
plt.tight_layout()
plt.savefig('ddpm_vs_ddim_final.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n=== Summary ===")
print(f"DDPM: 1000 steps — highest quality, slower")
print(f"DDIM: 50 steps — nearly identical quality, 20x faster")
print(f"Score Connection: ε_θ(x_t, t) = -√(1-ᾱ_t) · ∇log q(x_t)")
print("Diffusion models and score-based models are two sides of the same coin!")

In [ ]:
#@title 🎧 Wrap-Up: Closing
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_19_closing.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")


## 9. Reflection and Next Steps

### Key Takeaways
1. DDPM sampling requires T=1000 steps — each step is one neural network forward pass
2. DDIM enables deterministic sampling in far fewer steps (50-100) with minimal quality loss
3. Noise prediction is mathematically equivalent to score function estimation
4. This unification reveals that DDPMs are performing discretized Langevin dynamics

### Reflection Questions
- Why is deterministic (DDIM) sampling useful beyond just speed? Think about image editing applications.
- The score function points toward high-probability regions. How does this help generate realistic images?
- How would you extend DDPM to generate images conditioned on text descriptions (like DALL-E)?
- What limits current diffusion models — is it the architecture, the noise schedule, or the sampling method?

### What Comes Next
The DDPM framework we built here is the foundation for modern image generation systems:
- **DALL-E 2:** DDPM + CLIP text embeddings for text-to-image generation
- **Stable Diffusion:** DDPM in the latent space of a VAE for efficiency
- **Imagen:** Cascaded DDPMs at increasing resolutions for high-fidelity output

The core principle remains the same: learn to predict noise, then iteratively denoise.